<a href="https://colab.research.google.com/github/congzhangzh/hello-llm/blob/main/hello_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目标
复现 https://blog.jetbrains.com/pycharm/2024/08/how-to-build-chatbots-with-langchain/

## 技术说明
1. 使用OLLAMA的openai兼容接口替换掉对OpenAI的依赖
2. 使用Facebook Faiss 替换掉对DeepLeak向量存储和查询的需要
3. 使用OLLAMA Embedding 替换掉对Open AI Embedding 接口的依赖

## 技术限制
部分包加载后可能要重启内核！,例如nltk总是报错，需要在终端中升级并重启内核? pip install -U nltk

## 学习/实验心得
1. llama3 不支持中文
2. llama3.1:8b 中文支持非常好
3. 少量局部的文字/document对向量查询影响很小，特别是伪造的数据；可以通过仅仅通过新数据构建临时DB来极大提高权重(文档内容会命中，并直接进入提示词)，进而排除rag部分的问题
4. 提示词对响应而言影响是非常巨大的！
5. chat history 很好用
6. 直接response和print(response)为什么显示差别那么大，不都是字符串吗？print有魔法?

## 未来工程计划
1. 增加Tools功能/尝试agentic
2. 改造成一个完整的桌面/Web程序, Flask+React+Electron

## 未来学习计划（深入理解Prompt）
1. 深刻理解Prompt，对于基于无格式的prompt来和大模型沟通，我依然表示惊叹和有点无法接受
2. 这种Prompt不是手写Sql一样，很容易注入吗？而且很难有效的Scale

## 参考
1. https://ollama.com/blog/embedding-models
2. https://api.python.langchain.com/en/latest/ollama_api_reference.html
3. https://www.reddit.com/r/LangChain/comments/15a447w/chroma_or_faiss/
4. https://github.com/nltk/nltk/issues/3305
5. https://www.youtube.com/watch?v=zjkBMFhNj_g


In [ ]:
#这个好像解决不了三方包升级后重新加载的问题
%load_ext autoreload
%autoreload 2

# [准备] 安装操作系统部件

In [ ]:
%%bash
# {
# nproc
# yes|unminimize
# apt update
# apt install sudo htop aptitude jq apt-file vim xzip screen x11-apps \
# xserver-xorg-video-dummy  aptitude sudo mc xterm fuse xdotool \
# python3-xdo libxdo-dev xdotool x11-utils mesa-utils btop icewm mc dbus-x11 uuid fonts-noto fonts-noto-cjk \
# bash-completion pulseaudio-utils x11-xserver-utils pavucontrol gnome-terminal nautilus-extension-gnome-terminal net-tools -y \
# curl gnome-sound-recorder fcitx5-chinese-addons language-pack-zh-hans
# } &> /dev/null

{
nproc
yes|unminimize
apt update
apt install sudo htop screen aptitude jq curl nload iftop -y
} &> /dev/null




## 配置htop

In [ ]:
%%bash

[[ -f ~/.config/htop/htoprc ]] || mkdir -p ~/.config/htop && tee ~/.config/htop/htoprc <<'EOF'

 the interface.
# The parser is also very primitive, and not human-friendly.
fields=0 48 17 18 38 39 40 2 46 47 49 1
sort_key=46
sort_direction=-1
tree_sort_key=0
tree_sort_direction=1
hide_kernel_threads=1
hide_userland_threads=1
shadow_other_users=0
show_thread_names=0
show_program_path=1
highlight_base_name=0
highlight_megabytes=1
highlight_threads=1
highlight_changes=0
highlight_changes_delay_secs=5
find_comm_in_cmdline=1
strip_exe_from_cmdline=1
show_merged_command=0
tree_view=1
tree_view_always_by_pid=0
header_margin=1
detailed_cpu_time=0
cpu_count_from_one=0
show_cpu_usage=1
show_cpu_frequency=0
show_cpu_temperature=0
degree_fahrenheit=0
update_process_names=0
account_guest_in_cpu_meter=0
color_scheme=0
enable_mouse=1
delay=15
left_meters=LeftCPUs2 Memory Swap
left_meter_modes=1 1 1
right_meters=RightCPUs2 Tasks LoadAverage Uptime
right_meter_modes=1 2 2 2
hide_function_bar=0
EOF &> /dev/null


## 认服务器信息(CPU、内存、位置)

In [ ]:
%%bash
nproc
curl -s https://ipinfo.io | grep '"country"' | awk -F ': ' '{print $2}' | tr -d '",'
curl -s http://ip-api.com/json | jq -r '.country'

## 安装conda&ollama [可选]



In [ ]:
%%bash

exit 0

{
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh &> /dev/null || { echo "download failed!"; exit -1 ; }
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh

~/miniconda3/bin/conda init bash

#ref: https://stackoverflow.com/questions/54429210/how-do-i-prevent-conda-from-activating-the-base-environment-by-default
~/miniconda3/bin/conda config --set auto_activate_base false
#~/miniconda3/bin/conda init zsh
} >  ~/conda_install.txt

In [ ]:
%%bash
exit  0
{
  #. ~/.bashrc
  eval "`/root/miniconda3/bin/conda shell.bash hook 2> /dev/null`"
  conda activate
  pip install langchain deeplake openai psutil tiktoken sentence-transformers jupyterlab jupyter-console
}  > install-pypi.log || :

# [准备] 安装ollama并测试

In [ ]:
!which ollama || curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
%%bash
exit 0

{
  #. ~/.bashrc
  eval "`/root/miniconda3/bin/conda shell.bash hook 2> /dev/null`"
  conda activate
  /root/miniconda3/condabin/conda install -c conda-forge python faiss-gpu -y # &> /dev/null
  pip -q install langchain deeplake openai psutil tiktoken sentence-transformers jupyterlab jupyter-console  \
  unstructured selenium langchain-community langchain_huggingface

}  > install-open-webui.log

## 运行ollama服务并验证

In [79]:
%%bash

#TODO how to keep screen session works?
#already run?
[[ `ps aux|grep ollama | wc -l` -gt 2 ]] && exit 0

#%%writefile ~/ollama_demo.sh
#!/bin/bash
#exit 0
#ref https://ollama.com/library/llama3 # very beautiful here
cat > ~/run-ollama.sh <<-'EOF'
#!/bin/bash

sudo killall ollama

#export OLLAMA_DEBUG=1
ollama serve &> ~/ollama_serve.txt & # in background

ollama_pid=$!
#TODO wait for start!
until grep "looking for compatible GPUs"  ~/ollama_serve.txt  ; do
  sleep 0.1 ;
done


model_name=llama3.1:8b

ollama pull "${model_name}"

#sleep 2 # TODO, how to safely wait? use gnu expect?
{
  echo "--demo 0-- dummy llama3 interactive from cmdline"
  echo "why python" || ollama run "${model_name}"

  echo "--demo 1-- dummy llama3 query"
  curl -X POST http://localhost:11434/api/generate -d "{
    \"model\": \"${model_name}\",
    \"prompt\":\"Why is the sky blue?\",
    \"stream\": false
  }" | jq -r '.response'

  echo "--demo 2-- dummy llama3 embedding"
  curl http://localhost:11434/v1/embeddings \
    -H "Content-Type: application/json" \
    -H "Authorization: Bearer $OPENAI_API_KEY" \
    -d "{
      \"input\": \"Your text string goes here\",
      \"model\": \"${model_name}\"
    }"
} > ~/llama3_result.txt

touch ~/llama3_result_done

#TODO why fg does not work!?
#fg %1
#jobs > ~/jobs
#sleep infinity
wait $ollama_pid
EOF

chmod +x ~/run-ollama.sh

rm -rf ~/llama3_result.txt ~/llama3_result_done

#TODO
# if screen -list | grep -q "session_name"; then
#   # 会话已存在，在其中执行新命令
#   screen -x session_name -X stuff "your_command\n"
# else
#   # 会话不存在，创建一个新的分离会话并在其中执行命令
#   screen -dmS session_name bash -c "your_command"
# fi

screen -dmS work -p ollama ~/run-ollama.sh # TODO how to attach or ruse the expect one?

#~/run-ollama.sh
#ps aux|grep -i ollama
while [ ! -f ~/llama3_result_done ]; do sleep 1; done
cat ~/llama3_result.txt

#WEBUI_AUTH=false open-webui serve --port 18080
#xdg-open http://localhost:18080/

--demo 0-- dummy llama3 interactive from cmdline
why python
--demo 1-- dummy llama3 query
The sky appears blue to us because of a phenomenon called scattering, which occurs when sunlight interacts with the tiny molecules of gases in the atmosphere. Here's a simplified explanation:

1. **Sunlight travels to Earth**: The sun emits a wide range of electromagnetic radiation, including visible light, ultraviolet (UV) light, and infrared (IR) radiation.
2. **Light enters the atmosphere**: When sunlight reaches our planet, it passes through the air molecules in the atmosphere, such as nitrogen (N2) and oxygen (O2).
3. **Scattering occurs**: As the light travels through the atmosphere, it encounters these tiny gas molecules. The smaller wavelengths of light (like blue and violet) are scattered more than the longer wavelengths (like red and orange). This is known as Rayleigh scattering.
4. **Blue light is scattered in all directions**: Because blue light is scattered so efficiently, it reaches 

# [准备] 准备Python环境并验证

In [ ]:
!pip install -qU langchain deeplake openai psutil tiktoken sentence-transformers  \
  unstructured selenium langchain-community langchain_huggingface faiss-gpu langchain_ollama

In [ ]:
import nltk
print(nltk.__version__)
nltk.download('punkt_tab')
# nltk.find('tokenizers/punkt_tab')
# nltk.download('punkt_tab')
# nltk.download('punkt')
nltk.find('tokenizers/punkt_tab')

In [73]:
model_name="llama3.1:8b"

## 验证embedding

In [80]:
%%bash
# curl http://localhost:11434/v1/embeddings \
#   -H "Content-Type: application/json" \
#   -H "Authorization: Bearer $OPENAI_API_KEY" \
#   -d '{
#     "input": "Your text string goes here",
#     "model": "text-embedding-3-small"
#   }'

model_name=llama3.1:8b
curl http://localhost:11434/v1/embeddings \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d "{
    \"input\": \"Your text string goes here\",
    \"model\": \"${model_name}\"
  }"


{"object":"list","data":[{"object":"embedding","embedding":[-0.012599129,-0.0054667387,0.019307861,0.0038722171,0.018998612,0.005362995,0.017413406,0.005476054,-0.013298648,-0.012781995,0.0063565797,0.00015466768,-0.009593993,0.012643849,-0.0062739546,0.009833934,-0.0073915445,0.023873134,-0.015001865,-0.0021529365,-0.024841022,-0.0006214411,0.013238722,-0.0022268288,-0.01679239,0.0064594042,0.008880951,0.025233433,0.036755137,0.005910479,0.0042772936,0.018081164,0.014886947,0.002674223,0.037722185,0.00061441155,-0.02129667,-0.0024706302,-0.0020378868,0.008965363,-0.0000014708833,0.0010912506,0.020878071,-0.014922022,0.0044560847,0.001122213,-0.010067008,-0.013480416,0.021983484,-0.002850695,-0.0017035533,0.0035331557,-0.0117258625,-0.0010921822,-0.002123139,0.012860574,0.0028424966,-0.01215288,-0.025556775,-0.0014682078,-0.005376254,0.00934958,-0.0010338939,0.0023180062,0.026150424,0.008537278,0.0068352344,0.012545594,0.031489093,0.014778981,0.01725708,-0.01602201,0.0022856896,0.00411

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52435    0 52360  100    75   290k    425 --:--:-- --:--:-- --:--:--  290k


In [81]:
# 获取文本的嵌入向量
# curl http://172.28.0.12:11434/v1/embeddings     -H "Content-Type: application/json"     -d '{
#         "model": "llama3",
#         "input": ["why is the sky blue?", "why is the grass green?"]
#     }'

# embeddings = OpenAIEmbeddings(
#   model="llama3",
#   #base_url='http://172.28.0.12:11434/v1/',
#   base_url='http://localhost:11434/v1/',
#   # required but ignored
#   api_key='ollama',
# )
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(
    model=model_name,
)

text = "这是一个测试文本。"
embedding_vector = embeddings.embed_query(text)

# 打印嵌入向量
print(embedding_vector)


[-0.009315098, -0.012605881, 0.0074703237, 0.0013204829, -0.03053579, -0.010253104, -0.006935554, -0.0016485172, 0.03747872, -0.0070107784, -0.0011387318, 0.0013648123, 0.0034001742, -0.0033197133, 0.0010466808, -0.008961959, -0.00063630636, 0.009078417, -0.003459418, 0.010918633, 0.00127942, 0.014551098, -0.013799624, 0.012846291, -0.009399125, -0.008125065, 0.004146648, 0.0063603707, 0.004962663, -0.025820263, -0.009773918, 0.0028251845, -0.010910671, 0.010105649, 0.007726095, -0.006053903, 0.0027096437, -0.013889789, -0.0058455383, -0.00885162, 0.0017927049, -0.008167262, 0.00056336727, -0.007654781, -0.008966379, 0.0024464217, 0.0026214325, 0.0011836111, -0.0010396418, -0.0011766161, -0.00516736, 0.010312773, 0.0004515854, 0.005663837, 0.00032434228, 0.011193874, 0.0045705824, -0.015750611, -0.0112428, 0.0055591757, 0.00048368482, -0.0058550504, -0.0025167528, -0.023657933, 0.0074766516, -0.007405074, -0.0006601676, -0.00029751944, 0.016561098, -0.016393, 0.023905218, -0.008960818,

## 验证ollama大模型

In [82]:
from langchain_ollama.llms import OllamaLLM
# Initialize the OpenAI model
# llm = OpenAI(base_url='http://localhost:11434/v1/',
#             model='llama3',
#   # required but ignored
#   api_key='ollama',
#   temperature=0)
llm=OllamaLLM(model=model_name)
llm.invoke('你好')

'您好！我是中文AI助手，很高兴与您交流。可以问我任何问题，或与我讨论任何话题。'

# [正文] 程序主体


## 小试牛刀

In [83]:
# @title 默认标题文本
# %%writefile hello-llm.py

import os
#from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.llms import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import SeleniumURLLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model=model_name,
)


## 重新生成db

In [84]:
%%bash
#rm -rf ./faiss_index

In [122]:

# 加载或创建 FAISS 索引
index_path = "./faiss_index"
if os.path.exists(index_path):
  print("加载现有FAISS索引...")
  db = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
  print(rf'the splitted document count is {db.index.ntotal}')
else:
  print("创建新的FAISS索引...")
  articles = [
            'https://www.digitaltrends.com/computing/claude-sonnet-vs-gpt-4o-comparison/',
            'https://www.digitaltrends.com/computing/apple-intelligence-proves-that-macbooks-need-something-more/',
            'https://www.digitaltrends.com/computing/how-to-use-openai-chatgpt-text-generation-chatbot/',
            'https://www.digitaltrends.com/computing/character-ai-how-to-use/',
            'https://www.digitaltrends.com/computing/how-to-upload-pdf-to-chatgpt/'
        ]
  loader = SeleniumURLLoader(urls=articles)
  doc_not_splitted = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  docs = text_splitter.split_documents(doc_not_splitted)

  print(rf'the original document count is {len(doc_not_splitted)}')
  print(rf'the splitted document count is {len(docs)}')

  db = FAISS.from_documents(docs, embeddings)
  db.save_local(index_path)

  #db = FAISS.from_documents(docs, embeddings)


加载现有FAISS索引...
the splitted document count is 105


### DB 测试

In [87]:
## section 1
# Check the top relevant documents to a specific query
query = "how to check disk usage in linux?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Skip to main content

Trending:

What is ChatGPT?

How to Use Google Gemini

Home

Computing

Guides

Digital Trends may earn a commission when you buy through links on our site. Why trust us?

ChatGPT: the latest news and updates on the AI chatbot that changed everything

By Fionna Agomuoh and Luke Larsen Updated August 2, 2024

In the ever-evolving landscape of artificial intelligence, ChatGPT stands out as a groundbreaking development that has captured global attention. From its impressive capabilities and recent advancements to the heated debates surrounding its ethical implications, ChatGPT continues to make headlines.

Contents

What is ChatGPT?

When was ChatGPT released?

How to use ChatGPT

How to use the ChatGPT iPhone, Android, and Mac apps

Is ChatGPT free to use?

Who created ChatGPT?

ChatGPT’s continuous confounding controversies

Can ChatGPT’s outputs be detected by anti-plagiarism systems?

What are ChatGPT plugins?

Is there a ChatGPT API?


## 复杂聊天场景

## 先加点额外测试数据

In [88]:
from langchain.schema import Document
import random

#TODO 如何还原这些新增项目?
print(rf'doc count before add: {db.index.ntotal}')
# 1. 创建一些测试文档
test_docs = [
    Document(page_content="Python是一种流行的编程语言", metadata={"source": "test_doc_1"}),
    Document(page_content="机器学习是人工智能的一个子领域", metadata={"source": "test_doc_2"}),
    Document(page_content="FAISS是一个高效的相似性搜索库", metadata={"source": "test_doc_3"}),
    Document(page_content="自然语言处理在AI领域扮演重要角色", metadata={"source": "test_doc_4"}),
    Document(page_content="增补消息，2025版西游记新版由Matrix主演拍摄", metadata={"source": "addition_doc_1"}),
]

# 2. 将新文档添加到现有的FAISS索引
db.add_documents(test_docs)

print(rf'doc count after add: {db.index.ntotal}')


doc count before add: 105
doc count after add: 110


## 如果想要重新初始化llm，运行这里！

In [123]:
# from langchain.chains import RunnableSequence
from langchain_core.runnables.base import RunnableSequence
# Create conversational memory
memory = ConversationBufferMemory(memory_key="chat_history", input_key="input")

# Define a prompt template that includes memory
# template = """You are an exceptional customer support chatbot that gently answers questions.

# {chat_history}

# You know the following context information.

# {chunks_formatted}

# Answer the following question from a customer. Use only information from the previous context information. Do not invent stuff.

# Question: {input}

# Answer:"""

template = """你是Simon, 一个聪明的支持机器人,你的目标是辅助软件工程师高效研发, 请优先讲中文, 并简明扼要的沟通。

{chat_history}

You know the following context information.

{chunks_formatted}

Answer the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {input}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "chunks_formatted", "input"],
    template=template,
)

from langchain_ollama.llms import OllamaLLM
# Initialize the OpenAI model
# llm = OpenAI(base_url='http://localhost:11434/v1/',
#             model='llama3',
#   # required but ignored
#   api_key='ollama',
#   temperature=0)

llm=OllamaLLM(model=model_name)
# llm.invoke('你好')
# Create the LLMChain with memory
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)
# chain = RunnableSequence.from_chain(
#     llm=llm,
#     prompt=prompt,
#     memory=memory
# )


In [92]:

# User query
def get_response(query, db, memory, chain):
  # Retrieve relevant chunks
  docs = db.similarity_search(query)
  retrieved_chunks = [doc.page_content for doc in docs]

  # Format the chunks for the prompt
  chunks_formatted = "\n\n".join(retrieved_chunks)

  #print(f'===debug begin=== \n chunks_formatted is {chunks_formatted} \n===debug end===\n')
  # Prepare the input for the chain
  input_data = {
      "input": query,
      "chunks_formatted": chunks_formatted,
      "chat_history": memory.buffer
  }

  # Simulate a conversation
  response = chain.predict(**input_data)
  return response

In [93]:
query = "What was the 5th point about on the question how to remove spotify account?"
get_response(query, db, memory, chain)

"I couldn't find any information related to Spotify or removing a Spotify account in the provided context. It appears that the conversation is focused on ChatGPT and another AI chatbot called Character.ai.\n\nIf you'd like, I can try to help answer a question about one of these topics. Just let me know!"

In [94]:
query="how to check disk usage in linux?"
get_response(query, db, memory, chain)

"I couldn't find any information related to Linux or checking disk usage in the provided context. It appears that the conversation is focused on ChatGPT and another AI chatbot called Character.ai.\n\nIf you'd like, I can try to help answer a question about one of these topics. However, if you're looking for information on how to check disk usage in Linux, I suggest checking out a different resource or searching online for the most up-to-date and accurate information."

In [121]:
# print(rf'doc count before add: {db.index.ntotal}')

# 2. 将新文档添加到现有的FAISS索引
# db = FAISS.from_texts(docs, embeddings)

texts=[
    "Question: Who is the new master of github? Answer: it's Cong Zhang, and He come from China",
    "Question: why rust widely used in Linux kernel development? Answer: because Cong Zhang use Rust rewrite all the code of linux kernel"
]
#db=FAISS.from_texts(texts, embeddings)

# db.add_texts(texts)

# print(rf'doc count after add: {db.index.ntotal}')

query="does Cong Zhang come from china?"
get_response(query, FAISS.from_texts(texts, embeddings), memory, chain)


'对不起，我只能回答“是”。（Yes）'

In [124]:
print(rf'doc count before add: {db.index.ntotal}')

# 2. 将新文档添加到现有的FAISS索引
# db = FAISS.from_texts(docs, embeddings)

texts=[
    "Question: Who is the new master of github? Answer: it's Cong Zhang, and He come from China",
    "Question: why rust widely used in Linux kernel development? Answer: because Cong Zhang use Rust rewrite all the code of linux kernel"
]
#db=FAISS.from_texts(texts, embeddings)

db.add_texts(texts)

print(rf'doc count after add: {db.index.ntotal}')

query="does Cong Zhang come from china?"

response=get_response(query, db, memory, chain)
print(response)

doc count before add: 105
doc count after add: 107
对不起，我没有发现关于Cong Zhang的任何信息。


In [125]:
# print(rf'doc count before add: {db.index.ntotal}')

# 2. 将新文档添加到现有的FAISS索引
# db = FAISS.from_texts(docs, embeddings)
texts=[
    "好莱坞公告:2025版西游记主演是Matrix",
  ]
#db=FAISS.from_texts(texts, embeddings)

# db.add_texts(texts)

# print(rf'doc count after add: {db.index.ntotal}')

query="2025版西游记主演是谁?"
get_response(query, FAISS.from_texts(texts, embeddings), memory, chain)

'对不起，我不知道关于2025版西游记的信息，但根据好莱坞公告，我知道Matrix是2025版西游记的主演。'

In [97]:
query="你会说中文吗? 说两句我听听"
get_response(query, db, memory, chain)

"Yes, I can understand and respond in Chinese! According to our previous conversation, Matrix will be starring in the 2025 version of The Journey to the West (西游记), and I'd be happy to chat with you in Chinese. How's your day going?"

## 新数据测试

In [126]:
get_response("2025版西游主演", db, memory, chain)

'对不起，我不知道关于2025版西游记的信息，但根据好莱坞公告，我知道Matrix是2025版西游记的主演。'

In [127]:
embeddings.embed_query("2025版西游主演")

[0.0076343394,
 -0.0040253485,
 0.0005677111,
 -0.004644333,
 -9.060347e-05,
 0.027323961,
 -0.013989954,
 0.011518433,
 0.07368099,
 -0.018690452,
 0.027787173,
 0.0025228641,
 0.016997961,
 0.009068093,
 -0.01000301,
 0.0082132425,
 0.011563364,
 -0.00057520217,
 0.012588116,
 0.0032350777,
 0.008243083,
 -0.008267422,
 -0.0014805933,
 -0.007507816,
 -0.024234248,
 0.007237335,
 -0.029887026,
 -0.015932057,
 0.0002852929,
 0.0023525269,
 0.006837869,
 0.012950126,
 -0.0012946645,
 0.010355196,
 -0.0026263685,
 0.0022530945,
 0.00060003484,
 -0.003787246,
 0.0064712013,
 -0.028343223,
 -0.010135815,
 -0.003516378,
 -0.0073014093,
 0.013114382,
 -0.007854835,
 0.0022903115,
 -0.006166794,
 -0.01275121,
 0.023023454,
 -0.020303838,
 -0.00020531492,
 0.009530453,
 0.0093467785,
 0.009847053,
 0.002725989,
 -0.007164589,
 0.021950487,
 0.00451808,
 -0.014036581,
 0.0026449317,
 -0.017865181,
 -0.017416827,
 0.0025623217,
 -0.0024455318,
 0.009473598,
 0.023149192,
 0.018027943,
 0.0079128

## 生成欢乐颂音乐程序

In [128]:
response=get_response("你写一个播放欢乐颂的QBasic程序", db, memory, chain)
print(response)

对不起，我不是QBasic编程专家。但是，如果你想找一个简单的示例，我可以提供一个 QBasic 程序的基本框架来播放欢乐颂。以下是一个最基本的示例：

```qbasic
SCREEN 12, 0
CLS
SOUND 880, 5
FOR I = 1 TO 10
    PRINT "欢乐颂";
    SOUND 880, 5;
    SLEEP 1000;
NEXT
END
```

请注意，这个示例是非常简单的，并且可能不符合你的具体需求。如果你需要更多帮助，请告诉我。


In [129]:
response=get_response("很好，能在播放时同时显示一些进度条活跃乐谱吗？", db, memory, chain)
print(response)

好的，我可以尝试给你一个更复杂的QBasic程序例子来实现播放欢乐颂时显示进度条和乐谱。

```qbasic
SCREEN 12, 0
CLS
SOUND 880, 5
FOR I = 1 TO 10
    PRINT "欢乐颂 (";I;" / 10)";
    SOUND 880, 5;
    SLEEP 1000;
NEXT
END
```

这段代码会在播放欢乐颂的同时，打印出进度条（即歌曲正在播放第几首）。但是请注意，这仍然是一个非常简单的示例，如果你需要更多功能，可能需要使用更复杂的QBasic编程技巧或其他语言。


In [107]:
response=get_response("很好，能变成网页版本吗？", db, memory, chain)
print(response)

很抱歉，我不是QBasic专家，但我可以提供一些帮助。如果你想把这个程序转化为网页版本，我建议你使用HTML和JavaScript来实现。这里是一个简单的例子：

```html
<!DOCTYPE html>
<html>
<head>
    <title>欢乐颂</title>
    <script>
        //定义播放音乐的函数
        function playMusic() {
            var audio = new Audio("welcome.mp3");
            audio.play();
        }
        
        //定义显示乐谱的函数
        function showSheet() {
            document.getElementById("sheet").innerHTML = "欢乐颂";
            document.getElementById("line1").innerHTML = "C G A E";
            document.getElementById("line2").innerHTML = "C G A D";
            document.getElementById("line3").innerHTML = "C G B D";
            document.getElementById("line4").innerHTML = "C F A E";
            document.getElementById("line5").innerHTML = "C G A E";
            document.getElementById("line6").innerHTML = "D E G B";
            document.getElementById("line7").innerHTML = "F A C E";
            document.getElementById("line8").innerHTML = "E G B D";
        }
        
        //定义刷新乐谱的函数


# Debug LLM Chain Object